In [17]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [18]:
FeatureSetB_Raw = pd.read_csv('ProcessedFiles2/FeatureSetB_Raw.csv')

In [19]:
def DaysFilter(Y):
    #D2 = Y.date.isin(np.array([pd.Timestamp(Nextdate)]).astype('datetime64[ns]'))
    Starttime = Y.time[0]
    Startdate = Y.date[0]
    Nextdate = Startdate + timedelta(days=1)
    Lastdate = Nextdate  + timedelta(days=1)

    D1 = Y.date.isin(np.array([Startdate]))
    D2 = Y.date.isin(np.array([Nextdate]))
    D3 = Y.date.isin(np.array([Lastdate]))
    Y1 = Y[D1].copy()
    Y2 = Y[D2].copy()
    Y3 = Y[D3].copy()
   

    for i in Y3.index:
        if(Y3['time'][i] <= Starttime):
            continue
        else:
            Y3.drop(i, inplace=True)
        
    Yt = [Y1,Y2,Y3]
    Y4 = pd.concat(Yt)
    return Y4

In [20]:
import progressbar 
FeatureSetB = pd.DataFrame()
FeatureSetB['Date'] = pd.to_datetime(FeatureSetB_Raw.date, format='%Y-%m-%d')
FeatureSetB['time'] = pd.to_datetime(FeatureSetB_Raw.time, format='%H:%M:%S')
FeatureSetB['date'] = pd.to_datetime(FeatureSetB['Date']).dt.date
FeatureSetB['Time'] = pd.to_datetime(FeatureSetB['time']).dt.time

FeatureSetB_Raw['date'] = FeatureSetB['date']
FeatureSetB_Raw['time'] = FeatureSetB['Time']

FeatureSetB_Raw.sort_values(['subject_id','date','time'], ascending=[True, True, True],inplace=True)
FeatureSetB_Raw.reset_index(drop=True,inplace=True)

In [21]:
Subject_id = pd.DataFrame()
Subject_id = FeatureSetB_Raw.subject_id
Subject_id.drop_duplicates(keep = 'first', inplace = True)


In [22]:
widgets=[' [', progressbar.Timer(), '] ',progressbar.Percentage(),progressbar.Bar(),' (', progressbar.ETA(), ') ',]
FeatureSet = pd.DataFrame()
for X in progressbar.progressbar(Subject_id, widgets=widgets):
    Y = FeatureSetB_Raw.loc[FeatureSetB_Raw['subject_id'] == X]
    #print('Value of X',X)
    Y.reset_index(drop=True,inplace=True)
    Value = DaysFilter(Y)
    FeatureSet = pd.concat([FeatureSet,Value])


 [Elapsed Time: 0:00:41] 100%|##############################| (Time:  0:00:41) 


In [23]:
FeatureSet.reset_index(drop=True,inplace=True)

In [24]:
FeatureSet.to_csv('FeatureSet/FS_48Hours.csv', encoding='utf-8')